In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import glob
from datetime import datetime as dt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

In [2]:
# df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

In [3]:
df_sf_2017 = pd.read_json('df_sf_2017')

In [54]:
# df_sf['price'].apply(np.log)[df_sf_2017['price_per_guest'].isnull()][[ 'price', 'guests_included','price_per_guest']]

,price,guests_included,price_per_guest


In [3]:
df_sf_2017.price_per_bedroom.isnull().sum()

0

## Random Forest

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pickle

In [4]:
df_sf_2017.loc[df_sf_2017.cancellation_policy == 'flexible_new', 'cancellation_policy'] = "flexible"

In [46]:
df_sf_2017.property_type.value_counts()

Apartment             64831
House                 34244
Condominium            8291
Loft                   1634
Other                  1578
Bed & Breakfast        1128
Townhouse               877
Boutique hotel          812
Dorm                    699
In-law                  639
Guest suite             599
Timeshare               554
Guesthouse              342
Hostel                  222
Camper/RV               210
Bungalow                177
Cabin                   126
Boat                     78
Treehouse                40
Castle                   35
Villa                    32
Floor                    20
Serviced apartment       20
Lighthouse               15
Yurt                     14
Cave                     13
Tent                      9
Earth House               8
Hotel                     8
Pension (Korea)           3
Casa particular           2
Vacation home             1
Tipi                      1
Name: property_type, dtype: int64

## Predict 2017

In [5]:
def split_data(start_month, end_month):
    df_sf_temp = df_sf_2017.copy()
    X_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)][columns_to_keep]
    y_train = df_sf_temp[(df_sf_temp['month'] >= start_month) & (df_sf_temp['month'] < end_month)]['popular']

    X_test = df_sf_temp[df_sf_temp['month'] == end_month][columns_to_keep]
    y_test = df_sf_temp[df_sf_temp['month'] == end_month]['popular']
    
    return X_train, y_train, X_test, y_test
    

In [5]:
def transform_data(X):
    X['extra_people'].replace('[\=$,]', '', regex=True, inplace=True)
#     X['price'].replace('[\=$,]', '', regex=True, inplace=True)
    X['property_type'] = le.fit_transform(X['property_type'])
    X['bed_type'] = le.fit_transform(X['bed_type'])
    X['cancellation_policy'] = le.fit_transform(X['cancellation_policy'])
    X['room_type'] = le.fit_transform(X['room_type'])
 
    return X

In [6]:
def predict_data(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators = 1000, n_jobs=-1, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train, y_train)
    predicted = rf.predict(X_test)
    scores[0][model_num] = accuracy_score(y_test, predicted)
    scores[1][model_num] = recall_score(y_test, rf.predict(X_test))
    scores[2][model_num] = precision_score(y_test, rf.predict(X_test))
    scores[3][model_num] = f1_score(y_test, rf.predict(X_test))
    return scores

In [7]:
scores = np.zeros(shape=(4,9))
model_num = 0
start_month = 1
end_month = 4
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type']
# columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type']
columns_to_keep = ['access_filled', 'house_rules_filled','space_filled','accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type']

In [8]:
%%time
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    le = LabelEncoder()
    X_train = transform_data(X_train)
    X_test = transform_data(X_test)
    scores = predict_data(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores)

1 4 0
[[0.86021382 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.38664987 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.71728972 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.50245499 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
[[0.86021382 0.92659185 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.38664987 0.66729679 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.71728972 0.90358362 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.50245499 0.76766945 0.         0.         0.         0.
  0.         0.         0.        ]]
3 6 2
[[0.86021382 0.92659185 0.92838468 0.         0.         0.
  0.         0.         0.        ]
 [0.38664987 0.66729679 0.6875     0.         0.         0.
  0.         0.         0.        ]
 [0.71728972 0.90358

In [51]:
from tempfile import TemporaryFile
scores_rf_baseline = TemporaryFile()
np.save(scores_rf_baseline,scores)x

In [53]:
scores_rf_baseline.seek(0) # Only needed here to simulate closing & reopening file
np.load(scores_rf_baseline)

array([[0.81988506, 0.89212093, 0.88964655, 0.88999322, 0.88094975,
        0.88061452, 0.88234636, 0.88732079, 0.85575529],
       [0.2336272 , 0.47006931, 0.48648649, 0.4973822 , 0.47533875,
        0.49067661, 0.49973698, 0.53308824, 0.5445224 ],
       [0.51456311, 0.88075561, 0.87473002, 0.88601036, 0.88855117,
        0.93692777, 0.9047619 , 0.89664311, 0.86982759],
       [0.32135123, 0.61298274, 0.62524122, 0.63710879, 0.61935028,
        0.64405594, 0.64384954, 0.66864295, 0.66976435]])

In [9]:
accuracy_2017 = np.mean(scores[0])
recall_2017 = np.mean(scores[1])
precision_2017 = np.mean(scores[2])
f1_score_2017 = np.mean(scores[3])
print(accuracy_2017)
print(recall_2017)
print(precision_2017)
print(f1_score_2017)

0.9131723417963893
0.6583134251860404
0.8889285616059868
0.7545529482577731


In [20]:
df_sf_2017.dtypes == 'object'

access                               True
access_filled                       False
accommodates                        False
amenities                            True
availability_30                     False
availability_365                    False
availability_60                     False
availability_90                     False
bathrooms                           False
bed_type                             True
bedrooms                            False
beds                                False
calculated_host_listings_count      False
calendar_last_scraped               False
calendar_updated                     True
cancellation_policy                  True
city                                 True
cleaning_fee                         True
country                              True
country_code                         True
description                          True
experiences_offered                  True
extra_people                        False
first_review                      

In [18]:
df_sf_2017[columns_to_keep].head()

,access_filled,house_rules_filled,space_filled,accommodates,bed_type,extra_people,price_per_guest,price_per_bedroom,guests_included,host_about_filled,host_picture_url_filled,cancellation_policy,room_type,property_type
100000,True,False,True,3,Real Bed,8,2.406092,4.812184,1,True,True,moderate,Private room,Apartment
100001,False,False,True,2,Real Bed,0,2.561982,5.123964,1,True,True,flexible,Private room,Apartment
100002,True,False,True,5,Real Bed,0,1.221850,3.054624,4,True,True,moderate,Entire home/apt,Apartment
100003,False,False,False,4,Real Bed,0,3.656610,3.656610,1,False,True,flexible,Entire home/apt,Apartment
100004,False,False,False,2,Real Bed,0,2.488367,4.976734,1,True,True,flexible,Private room,House


# Logistic Regression

In [6]:
from sklearn.linear_model import LogisticRegression


In [7]:
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant


In [27]:
# columns_to_keep = ['access_filled', 'house_rules_filled','space_filled','accommodates','bed_type', 'extra_people', 'price_per_guest', 'price_per_bedroom','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', 'property_type']

columns_to_keep = ['accommodates','bed_type', 'extra_people','guests_included', 'cancellation_policy', 'room_type', 'property_type']





In [25]:
X_train.dtypes

accommodates                             int64
extra_people                             int64
price_per_guest                        float64
price_per_bedroom                      float64
guests_included                          int64
bed_type_Airbed                          uint8
bed_type_Couch                           uint8
bed_type_Futon                           uint8
bed_type_Pull-out Sofa                   uint8
bed_type_Real Bed                        uint8
cancellation_policy_flexible             uint8
cancellation_policy_moderate             uint8
cancellation_policy_strict               uint8
cancellation_policy_super_strict_30      uint8
cancellation_policy_super_strict_60      uint8
room_type_Entire home/apt                uint8
room_type_Private room                   uint8
room_type_Shared room                    uint8
property_type_Apartment                  uint8
property_type_Bed & Breakfast            uint8
property_type_Boat                       uint8
property_type

In [26]:
X_train.head()

,accommodates,extra_people,price_per_guest,price_per_bedroom,guests_included,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Boutique hotel,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Castle,property_type_Cave,property_type_Condominium,property_type_Dorm,property_type_Guesthouse,property_type_Hostel,property_type_House,property_type_Lighthouse,property_type_Loft,property_type_Other,property_type_Pension (Korea),property_type_Serviced apartment,property_type_Tent,property_type_Timeshare,property_type_Townhouse,property_type_Treehouse,property_type_Villa,property_type_Yurt
100000,3,8,2.406092,4.812184,1,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100001,2,0,2.561982,5.123964,1,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100002,5,0,1.221850,3.054624,4,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100003,4,0,3.656610,3.656610,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100004,2,0,2.488367,4.976734,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


array([[-0.06181344, -0.00183309,  0.16832884, -0.71154652, -0.45648909,
        -0.17492616, -0.01469664,  0.07137052, -0.42169738,  0.89955899,
         0.58362888, -1.48090394, -0.86687444, -0.7895355 , -0.28499834,
        -0.21175405, -0.29072232,  0.51053638, -0.98906315,  0.61906034,
         0.7975138 ,  1.33635965, -0.34389766, -0.43784466, -0.58386609,
        -0.75860163,  0.73508215,  0.84384082,  0.97994574,  0.01543787,
        -0.59858157, -0.34192192, -0.092331  , -0.0483936 , -0.15646548,
        -0.05409713, -1.48553727, -0.81736429,  0.43377868, -0.42774315,
        -0.13141241]])

In [47]:
X_train, y_train, X_test, y_test = split_data(1, 4)
X_train = transform_logistic(X_train, columns_to_keep)
X_test = transform_logistic(X_test, columns_to_keep)

model = LogisticRegression(n_jobs=-1, class_weight = {0:.95, 1:.05})
model.fit(X_train, y_train)
model.coef_

['accommodates', 'bed_type', 'extra_people', 'guests_included', 'cancellation_policy', 'room_type', 'property_type', 'bed_type_Airbed', 'bed_type_Couch', 'bed_type_Futon', 'bed_type_Pull-out Sofa', 'bed_type_Real Bed', 'cancellation_policy_flexible', 'cancellation_policy_moderate', 'cancellation_policy_strict', 'cancellation_policy_super_strict_30', 'cancellation_policy_super_strict_60', 'room_type_Entire home/apt', 'room_type_Private room', 'room_type_Shared room', 'property_type_Apartment', 'property_type_Bed & Breakfast', 'property_type_Boat', 'property_type_Boutique hotel', 'property_type_Bungalow', 'property_type_Cabin', 'property_type_Camper/RV', 'property_type_Castle', 'property_type_Cave', 'property_type_Condominium', 'property_type_Dorm', 'property_type_Guesthouse', 'property_type_Hostel', 'property_type_House', 'property_type_Lighthouse', 'property_type_Loft', 'property_type_Other', 'property_type_Pension (Korea)', 'property_type_Serviced apartment', 'property_type_Tent', 'pr

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


array([[-0.06017035, -0.00163431,  0.12448746, -0.6205904 , -0.36424738,
        -0.45263955, -0.37859236, -0.4260981 , -1.28063266,  0.01237846,
        -0.3051793 , -0.45979544, -0.20893886, -1.08947387, -0.61193352,
        -0.54076041, -0.53718078,  0.16343871, -0.14709187,  0.13588077,
         0.22514902,  0.42126893, -0.07417285, -0.04959084, -0.08183505,
        -0.90963812,  0.31951247,  0.32641532,  0.13763304, -0.19547491,
        -0.07026644, -0.49184285, -0.28305857, -0.00798729, -0.01682404,
        -0.00478723, -0.47453344, -0.62215456,  0.05563783, -0.04848795,
        -0.0121771 ]])

In [48]:
from math import exp

for i, coef in enumerate(model.coef_[0]):
    print("beta%d: %.5f" % (i + 1, exp(coef)))

beta1: 0.94160
beta2: 0.99837
beta3: 1.13257
beta4: 0.53763
beta5: 0.69472
beta6: 0.63595
beta7: 0.68482
beta8: 0.65305
beta9: 0.27786
beta10: 1.01246
beta11: 0.73699
beta12: 0.63141
beta13: 0.81144
beta14: 0.33639
beta15: 0.54230
beta16: 0.58231
beta17: 0.58439
beta18: 1.17755
beta19: 0.86321
beta20: 1.14555
beta21: 1.25251
beta22: 1.52389
beta23: 0.92851
beta24: 0.95162
beta25: 0.92142
beta26: 0.40267
beta27: 1.37646
beta28: 1.38599
beta29: 1.14755
beta30: 0.82244
beta31: 0.93215
beta32: 0.61150
beta33: 0.75348
beta34: 0.99204
beta35: 0.98332
beta36: 0.99522
beta37: 0.62218
beta38: 0.53679
beta39: 1.05721
beta40: 0.95267
beta41: 0.98790


In [8]:
def transform_logistic(X, columns_to_keep):

    df_temp = X[columns_to_keep]
    data_dummies = pd.DataFrame([])
    drop = []
    
    #dummify columns
    for column in columns_to_keep:
        if (df_temp[column].dtypes == 'O'):
            drop.append(column)
            dummies = pd.get_dummies(df_temp[column], prefix= column)
            data_dummies = pd.concat([data_dummies, dummies], axis=1)

    #join with original df
    df_temp = df_temp.join(data_dummies)

    print(list(df_temp.columns.values))
    #drop columns that were dummified
    df_temp.drop(drop, axis=1, inplace = True)

    return df_temp



In [49]:
for i, col in enumerate(list(X_train.columns.values)):
    print(i, col)
    
    

0 accommodates
1 extra_people
2 guests_included
3 bed_type_Airbed
4 bed_type_Couch
5 bed_type_Futon
6 bed_type_Pull-out Sofa
7 bed_type_Real Bed
8 cancellation_policy_flexible
9 cancellation_policy_moderate
10 cancellation_policy_strict
11 cancellation_policy_super_strict_30
12 cancellation_policy_super_strict_60
13 room_type_Entire home/apt
14 room_type_Private room
15 room_type_Shared room
16 property_type_Apartment
17 property_type_Bed & Breakfast
18 property_type_Boat
19 property_type_Boutique hotel
20 property_type_Bungalow
21 property_type_Cabin
22 property_type_Camper/RV
23 property_type_Castle
24 property_type_Cave
25 property_type_Condominium
26 property_type_Dorm
27 property_type_Guesthouse
28 property_type_Hostel
29 property_type_House
30 property_type_Lighthouse
31 property_type_Loft
32 property_type_Other
33 property_type_Pension (Korea)
34 property_type_Serviced apartment
35 property_type_Tent
36 property_type_Timeshare
37 property_type_Townhouse
38 property_type_Treehous

In [50]:
prediction = model.predict(X_test)

ValueError: X has 40 features per sample; expecting 41

In [ ]:
def predict_logistic():
    model = LogisticRegression(n_jobs = -1, class_weight = {0:.95, 1:.05})
    model.fit(X_train, y_train)
    model.coef_
#     prediction = model.predict(X_test)

In [21]:
X_const = add_constant(X_train, prepend=True)
logit_model = Logit(y_train, X_const).fit()

In [ ]:
logit_model.summary()

In [14]:
# Compute the change in odds ratio from a one unit change in each feature.

In [ ]:
from math import exp

for i, coef in enumerate(model.coef_[0]):
    print "beta%d: %.5f" % (i + 1, exp(coef))

In [ ]:
# Increasing the GRE score by 1 point increases the chance of getting in by a factor of 1.00189.